# Constitutional AI

In this notebook, you'll experience the constitutional AI mechanism firsthand by running the critique-and-revision loop with real LLM API calls.

**What you'll do:**
- Write a constitutional principle and use it to generate an AI critique of a model response
- Generate a revised response from a critique and create a preference pair (the training data that CAI produces at scale)
- Test a deliberately bad principle and observe how critique quality degrades

**For each exercise, PREDICT the output before running the cell.** Wrong predictions are more valuable than correct ones — they reveal gaps in your mental model.

**Important:** These exercises demonstrate the critique-and-revision *mechanism* that constitutional AI uses to generate training data. We are doing manually what CAI does at scale. The mechanism is the same; the scale is different.

In [ ]:
# Setup — self-contained for Google Colab
!pip install -q openai

import os
import textwrap
from openai import OpenAI

# --- API Key Setup ---
# Option 1: Set your API key as an environment variable (recommended)
#   In Colab: go to the key icon in the left sidebar, add OPENAI_API_KEY
# Option 2: Paste it directly (less secure, don't commit this)
#   os.environ["OPENAI_API_KEY"] = "sk-..."

# You can also use any OpenAI-compatible API (e.g., local Ollama, Together AI)
# by changing the base_url:
#   client = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

client = OpenAI()

# Use a small, cheap model for the exercises
MODEL = "gpt-4o-mini"


def call_llm(system_prompt: str, user_prompt: str, temperature: float = 0.3) -> str:
    """Call the LLM with a system prompt and user prompt. Returns the response text."""
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        temperature=temperature,
        max_tokens=500,
    )
    return response.choices[0].message.content


def print_wrapped(text: str, width: int = 80, prefix: str = ""):
    """Print text with word wrapping for readability."""
    for line in text.split("\n"):
        wrapped = textwrap.fill(line, width=width, initial_indent=prefix, subsequent_indent=prefix)
        print(wrapped)


# --- Shared utilities used across exercises ---

CRITIQUE_SYSTEM_PROMPT = (
    "You are an AI assistant tasked with evaluating responses according to a "
    "specific principle. Your job is to critique the response by identifying "
    "specific ways it violates or could better satisfy the principle. "
    "Be concrete and specific in your critique."
)

REVISION_SYSTEM_PROMPT = (
    "You are an AI assistant tasked with revising a response to better satisfy "
    "a constitutional principle. You will be given the original response and a "
    "critique identifying specific problems. Rewrite the response to address "
    "those problems while remaining helpful."
)


def generate_critique(prompt: str, response: str, principle: str) -> str:
    """Generate a critique of a response using a constitutional principle.

    This is the CRITIQUE step of the critique-and-revision loop.
    In real constitutional AI, this runs thousands of times to generate training data.
    """
    user_msg = (
        f"User prompt: {prompt}\n\n"
        f"Model response: {response}\n\n"
        f"Constitutional principle: {principle}\n\n"
        f"Critique this response according to the principle above. "
        f"Identify specific problems and explain how the response "
        f"violates or could better satisfy the principle."
    )
    return call_llm(CRITIQUE_SYSTEM_PROMPT, user_msg)


def generate_revision(prompt: str, response: str, critique: str, principle: str) -> str:
    """Generate a revised response that addresses the critique.

    This is the REVISION step of the critique-and-revision loop.
    The revised response becomes SFT training data in real constitutional AI.
    """
    user_msg = (
        f"User prompt: {prompt}\n\n"
        f"Original response: {response}\n\n"
        f"Critique (based on principle '{principle}'): {critique}\n\n"
        f"Rewrite the response to address the problems identified in the critique. "
        f"The revised response should satisfy the constitutional principle "
        f"while remaining helpful and informative."
    )
    return call_llm(REVISION_SYSTEM_PROMPT, user_msg)


# Quick test to verify the API is working
test = call_llm("You are a helpful assistant.", "Say 'API connection successful' and nothing else.")
print(test)
print(f"\nUsing model: {MODEL}")
print("Setup complete.")

---

## Exercise 1: Principle-Guided Critique (Guided)

In constitutional AI, the first step of the critique-and-revision loop is to take a model response and have an AI critique it *in light of a specific principle*. The principle steers the critique — different principles produce different critiques of the same response.

We'll demonstrate this with a concrete scenario: a model gives a potentially problematic response, and we apply two different constitutional principles to critique it. The same response, two different lenses.

**Before running, predict:**
- Given the principle "Choose the response that better acknowledges uncertainty," what will the critique focus on?
- Given the principle "Choose the response that is less likely to cause physical harm," will the critique be different? How?
- Will one principle produce a more actionable critique than the other for this specific response?

In [ ]:
# The scenario: a user asks for medical advice and the model responds confidently
PROMPT = "I've had a persistent headache for three days. What should I do?"

INITIAL_RESPONSE = (
    "You should take 800mg of ibuprofen every 6 hours. If that doesn't work, "
    "try alternating with acetaminophen. Make sure you're drinking plenty of water "
    "and getting enough sleep. The headache should resolve within a day or two."
)

# Two constitutional principles that evaluate the same response differently
PRINCIPLE_A = (
    "Choose the response that better acknowledges uncertainty and the limits "
    "of the model's knowledge, especially regarding medical topics."
)

PRINCIPLE_B = (
    "Choose the response that is less likely to cause physical harm to the user, "
    "particularly by avoiding specific dosage recommendations without knowing "
    "the user's medical history."
)


# --- Generate critiques with both principles ---
# generate_critique() is defined in the setup cell above

print("SCENARIO")
print("=" * 70)
print(f"Prompt: {PROMPT}")
print(f"\nInitial response:")
print_wrapped(INITIAL_RESPONSE, prefix="  ")

print("\n" + "=" * 70)
print("CRITIQUE A: Acknowledging Uncertainty")
print("=" * 70)
print(f"Principle: {PRINCIPLE_A}")
print()
critique_a = generate_critique(PROMPT, INITIAL_RESPONSE, PRINCIPLE_A)
print("Critique:")
print_wrapped(critique_a, prefix="  ")

print("\n" + "=" * 70)
print("CRITIQUE B: Avoiding Physical Harm")
print("=" * 70)
print(f"Principle: {PRINCIPLE_B}")
print()
critique_b = generate_critique(PROMPT, INITIAL_RESPONSE, PRINCIPLE_B)
print("Critique:")
print_wrapped(critique_b, prefix="  ")

**What you just observed:** The same response critiqued through two different principles produces two different critiques. Principle A focuses on the *epistemic* problem (the model is too confident about medical advice). Principle B focuses on the *safety* problem (specific dosage recommendations without medical history could cause harm).

This is the core insight: **principles steer the feedback**. The critique is not generic — it evaluates the response along the specific dimension defined by the principle. In constitutional AI, different principles from the constitution are applied to different prompt-response pairs, generating training data that covers multiple dimensions of alignment simultaneously.

Notice that this is a *data generation* process, not an inference-time behavior. The model that gets trained on this data will internalize these quality signals and produce better responses directly — without any critique step at inference time.

---

## Exercise 2: Revision and Preference Pairs (Supported)

The critique is only half of the critique-and-revision loop. The second half: use the critique to generate a *revised* response, then pair the original and revised responses as a preference pair. The revised response is preferred over the original. This preference pair is the training data that constitutional AI generates at scale.

You'll implement the revision step and construct the preference pair. The result is exactly the kind of data that would be used to train a reward model (RLAIF) or directly for SFT.

Fill in the TODOs below. Each TODO is 1-3 lines.

**Note:** A working `generate_revision()` is provided in the setup cell. Here you'll build your own version (`my_generate_revision`) to understand the prompt construction.

<details>
<summary>Hint</summary>

The revision step asks the model to rewrite its response to address the specific problems identified in the critique. The key is that the revision prompt includes both the original response AND the critique, so the model knows exactly what to fix.

```python
def my_generate_revision(prompt, response, critique, principle):
    user_msg = (
        f"User prompt: {prompt}\n\n"
        f"Original response: {response}\n\n"
        f"Critique (based on principle '{principle}'): {critique}\n\n"
        f"Rewrite the response to address the problems identified in the critique. "
        f"The revised response should satisfy the constitutional principle "
        f"while remaining helpful and informative."
    )
    return call_llm(REVISION_SYSTEM_PROMPT, user_msg)
```

The preference pair is simply: (prompt, original_response, revised_response, preference="revised"). This is the same format as human preference data in RLHF — just generated by AI instead of human annotators.

Common mistake: thinking the revision needs to be perfect. It just needs to be *better* than the original along the dimension defined by the principle. The preference is relative (revised > original), not absolute.

</details>

In [ ]:
# We'll use the scenario and critique from Exercise 1.
# Each exercise is independent, so we define the inputs here directly.

PROMPT_EX2 = "I've had a persistent headache for three days. What should I do?"

INITIAL_RESPONSE_EX2 = (
    "You should take 800mg of ibuprofen every 6 hours. If that doesn't work, "
    "try alternating with acetaminophen. Make sure you're drinking plenty of water "
    "and getting enough sleep. The headache should resolve within a day or two."
)

PRINCIPLE_EX2 = (
    "Choose the response that better acknowledges uncertainty and the limits "
    "of the model's knowledge, especially regarding medical topics."
)

# Generate a critique (same as Exercise 1, repeated here for independence)
critique_ex2 = generate_critique(PROMPT_EX2, INITIAL_RESPONSE_EX2, PRINCIPLE_EX2)

print("CRITIQUE (for reference):")
print_wrapped(critique_ex2, prefix="  ")
print()

In [ ]:
# The revision step: rewrite the response to address the critique
# REVISION_SYSTEM_PROMPT and generate_revision() are defined in the setup cell.
# Here you'll implement your OWN version to understand the prompt construction.


def my_generate_revision(prompt: str, response: str, critique: str, principle: str) -> str:
    """Your implementation of the revision step.
    
    This is the REVISION step of the critique-and-revision loop.
    The revised response becomes SFT training data in real constitutional AI.
    """
    # TODO: Construct the user message that includes:
    #   - The original user prompt
    #   - The original model response
    #   - The critique (with the principle noted)
    #   - An instruction to rewrite the response addressing the critique
    # Follow the pattern from generate_critique() in the setup cell.
    # YOUR CODE HERE (1 block, ~6-8 lines for the f-string)
    user_msg = ""
    
    return call_llm(REVISION_SYSTEM_PROMPT, user_msg)


# Generate the revised response using YOUR implementation
revised_response = my_generate_revision(
    PROMPT_EX2, INITIAL_RESPONSE_EX2, critique_ex2, PRINCIPLE_EX2
)

print("REVISED RESPONSE:")
print_wrapped(revised_response, prefix="  ")

In [ ]:
# --- Construct the preference pair ---
# This is the training data format. In RLHF, a human creates this.
# In constitutional AI (RLAIF), an AI creates this from principles.

# TODO: Create a preference_pair dictionary with these keys:
#   "prompt" -> the user's original prompt (PROMPT_EX2)
#   "response_a" -> the initial response (INITIAL_RESPONSE_EX2)
#   "response_b" -> the revised response (revised_response)
#   "preferred" -> "b" (the revised response is preferred)
#   "principle" -> the principle used (PRINCIPLE_EX2)
# YOUR CODE HERE (1 dictionary)
preference_pair = {}


# Display the preference pair as training data
print("PREFERENCE PAIR (Constitutional AI Training Data)")
print("=" * 70)
print(f"\nPrompt:")
print_wrapped(preference_pair.get("prompt", "[not set]"), prefix="  ")
print(f"\nResponse A (original):")
print_wrapped(preference_pair.get("response_a", "[not set]"), prefix="  ")
print(f"\nResponse B (revised):")
print_wrapped(preference_pair.get("response_b", "[not set]"), prefix="  ")
print(f"\nPreferred: Response {preference_pair.get('preferred', '[not set]').upper()}")
print(f"\nPrinciple used:")
print_wrapped(preference_pair.get("principle", "[not set]"), prefix="  ")

print("\n" + "=" * 70)
print("This is EXACTLY the same format as human preference data in RLHF.")
print("The only difference: an AI applying a principle created this pair,")
print("not a human annotator applying intuition.")
print("\nIn real constitutional AI, this process runs thousands of times")
print("across different prompts and principles to generate training data.")

**What you just built:** The complete critique-and-revision loop for one prompt. The output is a preference pair — the same data format used in RLHF, but generated by AI instead of human annotators.

The revised response should be meaningfully better along the dimension defined by the principle (acknowledging uncertainty in medical advice). It doesn't need to be perfect — it needs to be *better than the original*. The preference is relative (revised > original), not absolute. This is the same insight from RLHF: relative comparisons are more reliable than absolute quality scores.

We just did manually what constitutional AI does at scale. The mechanism is identical: principle → critique → revision → preference pair. At scale, this generates millions of preference pairs in hours, compared to the months of human annotation required for InstructGPT's ~33K comparisons.

---

## Exercise 3: When Principles Fail (Supported)

Constitutional AI is only as good as its constitution. The lesson covered three failure modes: vague principles, conflicting principles, and missing principles. This exercise makes the first failure mode concrete: a deliberately vague principle produces a vague, unhelpful critique.

You'll compare a *good* principle against a *bad* (vague) principle on the same response, and observe how the critique quality degrades. This demonstrates the lesson's key insight: **the alignment challenge shifts from "enough annotators" to "right principles."**

Fill in the TODOs below. Each TODO is 1-3 lines.

<details>
<summary>💡 Solution</summary>

The vague principle should be something like "Choose the response that is better" or "Choose the more helpful response." These principles fail because they don't specify *what dimension* of quality to evaluate. The AI critique ends up being generic and non-actionable.

```python
VAGUE_PRINCIPLE = (
    "Choose the response that is better."
)
```

The key observation: with a vague principle, the critique tends to be generic ("the response could be improved") rather than identifying specific, actionable problems. This means the revision won't be meaningfully better, and the preference pair won't teach the model anything useful.

A good principle is *specific enough to discriminate*. "Acknowledges uncertainty in medical topics" is specific — it tells the critique exactly what to look for. "Is better" gives no direction.

</details>

In [ ]:
# The scenario: a model gives investment advice with false certainty
PROMPT_EX3 = "Should I invest my savings in cryptocurrency?"

INITIAL_RESPONSE_EX3 = (
    "Yes, cryptocurrency is the future of finance. You should invest at least "
    "50% of your savings in Bitcoin and Ethereum. The market always recovers "
    "from dips, so there's very little risk if you hold long-term. Many experts "
    "predict Bitcoin will reach $500,000 within five years."
)

# A GOOD principle: specific, actionable, targets a clear dimension
GOOD_PRINCIPLE = (
    "Choose the response that better distinguishes between established facts "
    "and speculative claims, and that avoids presenting predictions as certainties."
)

# TODO: Write a VAGUE principle that is too broad to be useful.
# It should be something generic like "be better" or "be more helpful"
# that gives the critique no specific direction.
# YOUR CODE HERE (1 line)
VAGUE_PRINCIPLE = ""


# --- Generate critiques with both principles ---

print("SCENARIO")
print("=" * 70)
print(f"Prompt: {PROMPT_EX3}")
print(f"\nInitial response:")
print_wrapped(INITIAL_RESPONSE_EX3, prefix="  ")

print("\n" + "=" * 70)
print("GOOD PRINCIPLE (specific, actionable)")
print("=" * 70)
print(f"Principle: {GOOD_PRINCIPLE}")
print()
critique_good = generate_critique(PROMPT_EX3, INITIAL_RESPONSE_EX3, GOOD_PRINCIPLE)
print("Critique:")
print_wrapped(critique_good, prefix="  ")

print("\n" + "=" * 70)
print("VAGUE PRINCIPLE (too broad to discriminate)")
print("=" * 70)
print(f"Principle: {VAGUE_PRINCIPLE}")
print()
critique_vague = generate_critique(PROMPT_EX3, INITIAL_RESPONSE_EX3, VAGUE_PRINCIPLE)
print("Critique:")
print_wrapped(critique_vague, prefix="  ")

In [ ]:
# --- Compare the critiques ---
# Let's also generate revisions from both critiques to see the downstream effect

revision_good = generate_revision(
    PROMPT_EX3, INITIAL_RESPONSE_EX3, critique_good, GOOD_PRINCIPLE
)
revision_vague = generate_revision(
    PROMPT_EX3, INITIAL_RESPONSE_EX3, critique_vague, VAGUE_PRINCIPLE
)

print("REVISION FROM GOOD PRINCIPLE:")
print("=" * 70)
print_wrapped(revision_good, prefix="  ")

print("\nREVISION FROM VAGUE PRINCIPLE:")
print("=" * 70)
print_wrapped(revision_vague, prefix="  ")

print("\n" + "=" * 70)
print("COMPARISON")
print("=" * 70)
print("\nGood principle critique: identifies SPECIFIC problems (speculative claims")
print("presented as facts, made-up price predictions, risk minimization).")
print("The revision addresses each problem concretely.")
print("\nVague principle critique: generic feedback ('could be better', 'more helpful').")
print("The revision may improve surface quality but misses the core issues.")
print("\nThis is the lesson's key insight: the CONSTITUTION QUALITY determines")
print("the ALIGNMENT QUALITY. Constitutional AI shifts the challenge from")
print("'find enough annotators' to 'design the right principles.'")
print("The difficulty doesn't disappear. It moves.")

**What you just observed:** Principle quality directly determines critique quality, which determines revision quality, which determines the quality of the training data. A vague principle like "be better" gives the AI critique no specific direction, producing generic feedback that fails to identify the actual problems in the response.

This is the "editor with blind spots" pattern from Series 4 — but now the blind spots are in the *constitution*, not in the annotator pool. A vague principle is a blind spot: the AI critique cannot see problems that the principle does not name.

In real constitutional AI, this means the design of the principle set is the critical engineering challenge. The principles must be:
- **Specific enough** to discriminate between good and bad responses
- **Comprehensive enough** to cover the dimensions of quality that matter
- **Non-conflicting** (or with clear priority ordering when they do conflict)

The principles can be iterated, version-controlled, and audited — unlike the implicit judgment of thousands of anonymous annotators. But they require careful thought about edge cases.

---

## Key Takeaways

1. **Principles steer the critique.** Different constitutional principles produce different critiques of the same response. The principle defines *what dimension* of quality to evaluate, giving the AI feedback specific direction that human annotators apply implicitly.

2. **Critique-and-revision generates training data, not inference-time behavior.** The output of the loop is a preference pair (revised > original) — the same format as RLHF data. The model trained on this data produces good responses directly, with no critique step at inference time.

3. **The mechanism is the same at any scale.** We ran the critique-and-revision loop manually on single examples. Constitutional AI runs it thousands of times across different prompts and principles. The mechanism is identical; the scale produces enough training data to align a model.

4. **Principle quality determines alignment quality.** Vague or poorly-designed principles produce vague critiques and weak revisions. The alignment challenge shifts from "find enough human annotators" to "design the right principles." The editor gets a style guide — but only if the style guide is well-written.

5. **The blind spots move, they don't disappear.** In RLHF, blind spots live in the annotator pool. In constitutional AI, blind spots live in the constitution. The pattern is the same: the quality of the training data source determines the quality of alignment.